In [1]:
import pandas as pd
import numpy as np

# Creating the dataset
data = [
    ['Technical', 'Senior', 'excellent', 'good', 'urban', 'yes'],
    ['Technical', 'Junior', 'excellent', 'good', 'urban', 'yes'],
    ['Non-Technical', 'Junior', 'average', 'poor', 'rural', 'no'],
    ['Technical', 'Senior', 'average', 'good', 'rural', 'no'],
    ['Technical', 'Senior', 'excellent', 'good', 'rural', 'yes']
]

columns = ['Role', 'Experience', 'Performance', 'InternetQuality', 'WorkLocation', 'Output']

df = pd.DataFrame(data, columns=columns)


X = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1])


def is_consistent(hypothesis, example):
    return all(h == '?' or h == e for h, e in zip(hypothesis, example))


def more_general(h1, h2):
    return all(h1[i] == '?' or h1[i] == h2[i] for i in range(len(h1)))


def candidate_elimination(X, y):
    n = X.shape[1]

    # Initialize S and G
    S = X[y == 'yes'][0].copy()
    G = [['?' for _ in range(n)]]

    for i in range(len(X)):
        if y[i] == 'yes':
            # Generalize S
            for j in range(n):
                if S[j] != X[i][j]:
                    S[j] = '?'

            # Remove inconsistent hypotheses from G
            G = [g for g in G if is_consistent(g, X[i])]

        else:  # Negative example
            new_G = []
            for g in G:
                if is_consistent(g, X[i]):
                    for j in range(n):
                        if S[j] != '?' and S[j] != X[i][j]:
                            new_h = g.copy()
                            new_h[j] = S[j]
                            if new_h not in new_G:
                                new_G.append(new_h)
                else:
                    new_G.append(g)

            # Keep only maximally general hypotheses
            G = [
                h for h in new_G
                if any(more_general(h, s) for s in [S])
            ]

            # Remove subsumed hypotheses
            G = [
                h for h in G
                if not any(
                    other != h and more_general(other, h)
                    for other in G
                )
            ]

        print(f"\nAfter instance {i+1}:")
        print("S =", S)
        print("G =", G)

    return S, G



final_S, final_G = candidate_elimination(X, y)

print("\nFinal Specific Hypothesis:", final_S)
print("Final General Hypothesis:", final_G)


After instance 1:
S = ['Technical' 'Senior' 'excellent' 'good' 'urban']
G = [['?', '?', '?', '?', '?']]

After instance 2:
S = ['Technical' '?' 'excellent' 'good' 'urban']
G = [['?', '?', '?', '?', '?']]

After instance 3:
S = ['Technical' '?' 'excellent' 'good' 'urban']
G = [['Technical', '?', '?', '?', '?'], ['?', '?', 'excellent', '?', '?'], ['?', '?', '?', 'good', '?'], ['?', '?', '?', '?', 'urban']]

After instance 4:
S = ['Technical' '?' 'excellent' 'good' 'urban']
G = [['?', '?', 'excellent', '?', '?'], ['?', '?', '?', '?', 'urban']]

After instance 5:
S = ['Technical' '?' 'excellent' 'good' '?']
G = [['?', '?', 'excellent', '?', '?']]

Final Specific Hypothesis: ['Technical' '?' 'excellent' 'good' '?']
Final General Hypothesis: [['?', '?', 'excellent', '?', '?']]


In [1]:
# Tuple to store student IDs
student_ids = ('S101', 'S102', 'S103', 'S104')

# Dictionary to store student academic details
students = {
    'S101': {'name': 'Asha', 'assignment': 78, 'test': 80, 'attendance': 92, 'hours': 8},
    'S102': {'name': 'Ravi', 'assignment': 65, 'test': 68, 'attendance': 85, 'hours': 5},
    'S103': {'name': 'Meena', 'assignment': 88, 'test': 90, 'attendance': 96, 'hours': 10},
    'S104': {'name': 'Kiran', 'assignment': 55, 'test': 58, 'attendance': 78, 'hours': 4}
}

# Function to calculate average score
def calculate_average(assignment, test):
    return (assignment + test) / 2

# Function to determine academic risk level
def determine_risk(avg_score, attendance, hours):
    if avg_score < 60 or attendance < 80 or hours < 5:
        return "High Risk"
    elif avg_score < 75:
        return "Moderate Risk"
    else:
        return "Low Risk"

# Create a list to store processed student records
report = []

for sid in student_ids:
    data = students[sid]
    avg = calculate_average(data['assignment'], data['test'])
    risk = determine_risk(avg, data['attendance'], data['hours'])
    
    report.append({
        'id': sid,
        'name': data['name'],
        'average': avg,
        'attendance': data['attendance'],
        'hours': data['hours'],
        'risk': risk
    })

# Sort the report by average score in decreasing order
report.sort(key=lambda x: x['average'], reverse=True)

# Display structured performance report
print("STUDENT PERFORMANCE REPORT (Sorted by Average Score)")
print("-" * 55)

for student in report:
    print(f"Student ID      : {student['id']}")
    print(f"Name            : {student['name']}")
    print(f"Average Score   : {student['average']:.2f}")
    print(f"Attendance (%)  : {student['attendance']}")
    print(f"Study Hours     : {student['hours']} hrs/week")
    print(f"Risk Level      : {student['risk']}")
    print("-" * 55)


STUDENT PERFORMANCE REPORT (Sorted by Average Score)
-------------------------------------------------------
Student ID      : S103
Name            : Meena
Average Score   : 89.00
Attendance (%)  : 96
Study Hours     : 10 hrs/week
Risk Level      : Low Risk
-------------------------------------------------------
Student ID      : S101
Name            : Asha
Average Score   : 79.00
Attendance (%)  : 92
Study Hours     : 8 hrs/week
Risk Level      : Low Risk
-------------------------------------------------------
Student ID      : S102
Name            : Ravi
Average Score   : 66.50
Attendance (%)  : 85
Study Hours     : 5 hrs/week
Risk Level      : Moderate Risk
-------------------------------------------------------
Student ID      : S104
Name            : Kiran
Average Score   : 56.50
Attendance (%)  : 78
Study Hours     : 4 hrs/week
Risk Level      : High Risk
-------------------------------------------------------


In [2]:
import re
from collections import defaultdict

VALID_ACTIVITIES = {"LOGIN", "LOGOUT", "SUBMIT_ASSIGNMENT"}

# Student class
class Student:
    def __init__(self, student_id, name):
        self.student_id = student_id
        self.name = name
        self.activities = []

    def add_activity(self, activity, date, time):
        self.activities.append((activity, date, time))

    def activity_summary(self):
        logins = sum(1 for a in self.activities if a[0] == "LOGIN")
        submissions = sum(1 for a in self.activities if a[0] == "SUBMIT_ASSIGNMENT")
        return logins, submissions


# Generator to read valid log entries
def read_log_file(filename):
    with open(filename, "r") as file:
        for line in file:
            try:
                parts = [p.strip() for p in line.split("|")]
                if len(parts) != 5:
                    raise ValueError("Invalid format")

                sid, name, activity, date, time = parts

                # Validate student ID
                if not re.match(r"S\d+", sid):
                    raise ValueError("Invalid Student ID")

                # Validate activity
                if activity not in VALID_ACTIVITIES:
                    raise ValueError("Invalid Activity")

                yield sid, name, activity, date, time

            except Exception:
                continue  # Ignore invalid entries


# Main processing
students = {}
daily_stats = defaultdict(int)
login_tracker = defaultdict(int)

for sid, name, activity, date, time in read_log_file("activity_log.txt"):
    if sid not in students:
        students[sid] = Student(sid, name)

    students[sid].add_activity(activity, date, time)
    daily_stats[date] += 1

    if activity == "LOGIN":
        login_tracker[sid] += 1
    elif activity == "LOGOUT":
        login_tracker[sid] -= 1


# Generate report
with open("activity_report.txt", "w") as report:
    print("STUDENT ACTIVITY REPORT")
    print("-" * 40)
    report.write("STUDENT ACTIVITY REPORT\n")

    for student in students.values():
        logins, submissions = student.activity_summary()
        line = f"{student.student_id} | Logins: {logins} | Submissions: {submissions}"
        print(line)
        report.write(line + "\n")

    print("\nABNORMAL BEHAVIOR (Multiple logins without logout):")
    report.write("\nABNORMAL BEHAVIOR:\n")
    for sid, count in login_tracker.items():
        if count > 0:
            msg = f"{sid} has {count} unclosed login(s)"
            print(msg)
            report.write(msg + "\n")

    print("\nDAILY ACTIVITY STATISTICS:")
    report.write("\nDAILY ACTIVITY STATISTICS:\n")
    for date, count in daily_stats.items():
        msg = f"{date} : {count} activities"
        print(msg)
        report.write(msg + "\n")


STUDENT ACTIVITY REPORT
----------------------------------------
S101 | Logins: 1 | Submissions: 0
S102 | Logins: 0 | Submissions: 1

ABNORMAL BEHAVIOR (Multiple logins without logout):

DAILY ACTIVITY STATISTICS:
2025-03-10 : 3 activities
